# Code for HW GHZ

#### imports

In [1]:
from qiskit import QuantumCircuit , ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit , transpile
from qiskit_aer import AerSimulator
import random

In [2]:
def print_raw_statevector ( state , label ) :
    print(f"\n{ label } (Raw Coefficients ):")
    print(state)

In [3]:
def print_statevector ( state , label ) :
    print(f"\n{ label }:")
    for i , amplitude in enumerate ( state ) :
    # Convert index to binary , pad to 5 qubits , and reverse for big -endian
        basis_state = f"|{ format (i, '05b')[:: -1]} >"
    # Only print the non - zero amplitudes
    if abs ( amplitude ) > 0.0001: # Filter out near - zero amplitudes
        print ( f"{ amplitude :.4 f} { basis_state }")

In [4]:
#let n be the number of qubits has to be 5 in this case
n=5
qc = QuantumCircuit(n)
creg = ClassicalRegister(2,'c')
qc.add_register(creg)
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)
rand_arr =[0,1,2]
random.shuffle(rand_arr)
qc.x(rand_arr[0])
qc.x(rand_arr[1])

In [5]:
initial_state = Statevector . from_instruction ( qc )
print_raw_statevector ( initial_state , " Initial GHZ State ( Coefficients )")
print_statevector (initial_state , " Initial GHZ State (Big - Endian Notation )")
qc.x(1)
error_state = Statevector . from_instruction ( qc )
print_raw_statevector ( error_state , " Error State ( Coefficients )")
print_statevector ( error_state , " Error State (Big - Endian Notation )")


 Initial GHZ State ( Coefficients ) (Raw Coefficients ):
Statevector([0.        +0.j, 0.70710678+0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j],
            dims=(2, 2, 2, 2, 2))

 Initial GHZ State (Big - Endian Notation ):

 Error State ( Coefficients ) (Raw Coefficients ):
Statevector([0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j, 0.70710678+0.j, 0.        +0.j,
             0.        +0.j, 0. 

In [6]:
qc.cx(0,3) # CNOT from qubit 0 to ancilla 1 ( stored in qubit 3)
qc.cx(1,3) # CNOT from qubit 1 to ancilla 1 ( stored in qubit 3)

# Parity check between qubit 0 and qubit 2 Z0Z2
qc . cx (0 , 4) # CNOT from qubit 0 to ancilla 2 ( stored in qubit 4)
qc . cx (2 , 4) # CNOT from qubit 2 to ancilla 2 ( stored in qubit 4)

# Step 4: Measure the ancilla qubits and store results in the classical register
qc.measure (3 , creg [0]) # Measure ancilla qubit 3 into classical bit 0 of creg
qc.measure (4 , creg [1]) # Measure ancilla qubit 4 into classical bit 1 of creg

# After the measurement , execute the circuit and retrieve classical bit values
# Use AerSimulator for simulation
simulator = AerSimulator ()

# Run the circuit and get the result
result = simulator.run(qc).result()

# Get the classical register values ( counts )
counts = result . get_counts ()

# Get the most common outcome ( assuming no noise , 1024 shots , etc .)
measured_bitstring = max ( counts , key = counts.get )

# Output the measured classical bits ( creg [0] and creg [1])
print(f" Measured classical register values : { counts }")
# Step 5: Apply error correction with double if statements
if measured_bitstring =='01': # creg [1] = 0 and creg [0] = 1
    qc . x (1)
if measured_bitstring == '11': # creg [1] = 1 and creg [0] = 1
    qc . x (0)
if measured_bitstring == '10': # creg [1] = 1 and creg [0] = 0
    qc . x (2)
# Step 6: Remove measurements and get final state
# Since we ’re using Statevector simulation , we need to remove the final measurements
final_state = Statevector.from_instruction(qc.remove_final_measurements(inplace = False ))
 # Print final state after error correction
print_raw_statevector ( final_state , " Final State ( Coefficients )")
print_statevector ( final_state , " Final State (Big - Endian Notation )")

 Measured classical register values : {'10': 1024}

 Final State ( Coefficients ) (Raw Coefficients ):
Statevector([0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.70710678+0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j],
            dims=(2, 2, 2, 2, 2))

 Final State (Big - Endian Notation ):


##### When looking at the output it is clear that the code is not able to correct for the two qubit flipping errors. This can be seen if the initial state is a logical |0,0,0> with a double bit flip is |1,0,1> if our two Stabilizers are Z1 and Z2  the second is Z2 and Z3. In this case the resulant parity is P(-1,-1) which means there is a bit flip on the second bit which actually is the exact opposite of the result that is expected and would say the resultant after correction would be |111>.